# Import necessary modules

We begin by importing all necessary modules. The install function can be used to install the uninstalled modules.

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
import urllib3
import json
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text

# Import Data From UK national grid

The data is from neso https://www.neso.energy/data-portal/historic-generation-mix/historic_gb_generation_mix 

In [2]:
http = urllib3.PoolManager()
query_url = "https://api.neso.energy/api/3/action/datastore_search_sql?sql=SELECT * from \"f93d1835-75bc-43e5-84ad-12472b180a98\""
resp = http.request("GET", query_url)
data = pd.DataFrame(json.loads(resp.data)['result']['records'])
len(data)


291592

# Preprocess dataframe

In [ ]:
cols = data.columns.tolist()
non_perc_cols = [col for col in cols if not col.endswith('_perc')]
perc_cols = [col for col in cols if col.endswith('_perc')]
cols = non_perc_cols + perc_cols
remove = ['DATETIME', 'CARBON_INTENSITY', '_full_text', '_id']
cols = ['CARBON_INTENSITY'] + [col for col in cols if col not in remove]
data_sort= pd.concat([
    data['DATETIME'].astype('datetime64[ns]'), 
    data[cols].astype(float)
    ], axis=1)
# data_sort = data[cols]
print(data_sort.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290924 entries, 0 to 290923
Data columns (total 34 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DATETIME          290924 non-null  datetime64[ns]
 1   CARBON_INTENSITY  290924 non-null  float64       
 2   NUCLEAR           290924 non-null  float64       
 3   STORAGE           290924 non-null  float64       
 4   LOW_CARBON        290924 non-null  float64       
 5   HYDRO             290924 non-null  float64       
 6   WIND_EMB          290924 non-null  float64       
 7   IMPORTS           290924 non-null  float64       
 8   GAS               290924 non-null  float64       
 9   COAL              290924 non-null  float64       
 10  FOSSIL            290924 non-null  float64       
 11  GENERATION        290924 non-null  float64       
 12  OTHER             290924 non-null  float64       
 13  ZERO_CARBON       290924 non-null  float64       
 14  BIOM

# Export to SQL

Create engine to postgreSQL

In [4]:
# PostgreSQL connection info
username = "postgres" 
password = "1234"
host = "localhost"          
port = "5432"               
database = "energy_forecast"  

# Create connection engine
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")


Export with df.to_sql

In [ ]:
# Export DataFrame to a table named "generation_data"
data_sort.to_sql("generation_data", engine, if_exists="replace", index=False)

print("Data exported to PostgreSQL successfully.")

✅ Data exported to PostgreSQL successfully.
